In [1]:
import argparse
import glob
import json
import multiprocessing
import os
import random
import re
from importlib import import_module
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, ConcatDataset, WeightedRandomSampler

from dataset import MaskBaseDataset
from loss import create_criterion
import copy
import wandb

In [2]:
wandb.login() # 각자 WandB 로그인 하기
# 9eee70600a60d9d41eecef494a78a696bd12d252

# 🐝 initialise a wandb run
wandb.init(
    project="EfficientNetV2s_YS", # 프로젝트 이름 "모델_버전_성명"
    config = {
    "lr": 0.0001,
    "epochs": 60,
    "batch_size": 64,
    "optimizer" : "Adam",
    "resize" : [384, 384],
    "criterion" : 'weight_cross_entropy'
    }
 )

# Copy your config 
config = wandb.config


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: o2rabbit (cv13). Use `wandb login --relogin` to force relogin


In [4]:
parser = argparse.ArgumentParser()

# Data and model checkpoints directories
parser.add_argument('--seed', type=int, default=42, help='random seed (default: 42)')
parser.add_argument('--epochs', type=int, default=config.epochs, help='number of epochs to train (default: 1)')
parser.add_argument('--dataset', type=str, default='MaskBaseDataset', help='dataset augmentation type (default: MaskBaseDataset)')
parser.add_argument('--preprocessing', type=str, default='Basepreprocessing', help='data augmentation type (default: Basepreprocessing)')
parser.add_argument('--RealAugmentation', type=str, default='RealAugmentation', help='data augmentation type (default: RealAugmentation)')
parser.add_argument("--resize", nargs="+", type=list, default=config.resize, help='resize size for image when training')
parser.add_argument('--batch_size', type=int, default=config.batch_size, help='input batch size for training (default: 64)')
parser.add_argument('--valid_batch_size', type=int, default=250, help='input batch size for validing (default: 1000)')
parser.add_argument('--model', type=str, default='efficientnet_v2_s', help='model type (default: BaseModel)')
parser.add_argument('--optimizer', type=str, default=config.optimizer, help='optimizer type (default: SGD)')
parser.add_argument('--lr', type=float, default=config.lr, help='learning rate (default: 1e-3)')
parser.add_argument('--val_ratio', type=float, default=0.2, help='ratio for validaton (default: 0.2)')
parser.add_argument('--criterion', type=str, default=config.criterion, help='criterion type (default: cross_entropy)')
parser.add_argument('--lr_decay_step', type=int, default=10, help='learning rate scheduler deacy step (default: 20)')
parser.add_argument('--log_interval', type=int, default=20, help='how many batches to wait before logging training status')
parser.add_argument('--name', default='exp', help='model save at {SM_MODEL_DIR}/{name}')

# Container environment
parser.add_argument('--data_dir', type=str, default=os.environ.get('SM_CHANNEL_TRAIN', '/opt/ml/input/data/train/images'))
parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR', './model'))

args = parser.parse_args()
print(args)

# data_dir = args.data_dir
# model_dir = args.model_dir

# train(data_dir, model_dir, args)
# wandb.finish()

usage: ipykernel_launcher.py [-h] [--seed SEED] [--epochs EPOCHS]
                             [--dataset DATASET]
                             [--preprocessing PREPROCESSING]
                             [--RealAugmentation REALAUGMENTATION]
                             [--resize RESIZE [RESIZE ...]]
                             [--batch_size BATCH_SIZE]
                             [--valid_batch_size VALID_BATCH_SIZE]
                             [--model MODEL] [--optimizer OPTIMIZER] [--lr LR]
                             [--val_ratio VAL_RATIO] [--criterion CRITERION]
                             [--lr_decay_step LR_DECAY_STEP]
                             [--log_interval LOG_INTERVAL] [--name NAME]
                             [--data_dir DATA_DIR] [--model_dir MODEL_DIR]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"b3cb22eb-d1a1-4df9-84df-ad4e7254548a" --shell=

SystemExit: 2

In [ ]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
seed_everything(args.seed)

save_dir = increment_path(os.path.join(model_dir, args.name))

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# -- dataset
dataset_module = getattr(import_module("dataset"), args.dataset)  # default: MaskBaseDataset
dataset = dataset_module(
    data_dir=data_dir,
)
num_classes = dataset.num_classes  # 18



# -- preprocessing --data_set
transform_module = getattr(import_module("dataset"), args.preprocessing)  # default: preprocessing
transform = transform_module(
    resize=args.resize,
    mean=dataset.mean,
    std=dataset.std,
)
dataset.set_transform(transform)

dataset_aug = copy.deepcopy(dataset)

# augmentation 적용
transform_module_aug = getattr(import_module("dataset"), args.RealAugmentation)  # default: RealAugmentation
transform_aug = transform_module_aug(
    resize=args.resize,
    mean=dataset_aug.mean,
    std=dataset_aug.std,
)
dataset_aug.set_transform(transform_aug)


train_set,val_set = dataset.split_dataset()